In [39]:
# matplotlib
import matplotlib.pylab as plt
import matplotlib as mpl
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

# statistics
import numpy as np
from scipy import signal

# object support
import copy

# time keeping
import time
from datetime import timedelta

# data munging
import random, string
import pandas as pd

# operating system
import os
import glob
# tensorflow
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# keras support
from keras.models import model_from_yaml
from keras.models import load_model
import Gaussian as g

In [34]:
dirnames = glob.glob('models/*')
dirname = dirnames[2]

fpaths = glob.glob(dirname + '/*')

h5_fpath = [fpath for fpath in fpaths if fpath.endswith('.h5')][0]
yaml_fpath =  [fpath for fpath in fpaths if fpath.endswith('.yaml')][0]

In [32]:
#Read  the model architecture from  YAML file
with open(yaml_fpath, 'r') as yaml_file:
    yaml_savedModel=yaml_file.read()
# Load the saved Yaml model
model_y= tf.keras.models.model_from_yaml(yaml_savedModel)
model_y.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 16)      160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 64)                0

In [37]:
# loading the model from the HDF5 file
model_h5 = model_y.load_weights(h5_fpath)

In [43]:
# model fitting
validation_split = 0.2
# create gaussian dataset
im_size=128
set_size=1000
dataset = g.create_set(im_size=im_size,set_size=set_size)
x_train, y_train = g.load_dataset(dataset)
batch_size = len(dataset)


Dataset loaded.
Image input shape: (4000, 128, 128, 1)
Label input shape: (4000, 3)


In [46]:
model = model_y
epochs = 20

split_at = int(x_train.shape[0] * (1-validation_split))
validation_x = x_train[split_at:]
validation_y = y_train[split_at:]
validation_data=(validation_x, validation_y)
print("\n***LEARNING START***")
start = time.time()
history = model.fit(x=x_train[:split_at],
                    y=y_train[:split_at], 
                    epochs=epochs, 
                    batch_size=batch_size,
                    validation_data=(validation_x, validation_y),
                    verbose=2)
print("***LEARNING END***")
elapsed = time.time() - start
print("\nTIME:",str(timedelta(seconds=elapsed)))


***LEARNING START***


RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.